In [1]:
import json
import pickle
import random
import redis
import glob
import os
from tqdm.notebook import tqdm
from collections import defaultdict
from functools import partial
from heapq import heappush, heappop

import argparse
import json
from multiprocessing import Pool

from transformers import AutoTokenizer

In [2]:
# loading redis
redisd = redis.Redis(host='localhost', port=6379, decode_responses=True)
redisd.flushall()

True

# Building entity index

In [3]:
# building redis entity index: {doc: {ent_id: passage containing this entity}}
pbar = tqdm(total=258079)
with open("../data/rawdata/popular_page_ent_link.jsonl") as f:
    line = f.readline()
    while line:
        data = json.loads(line)
        doc_title = data['title']
        ent_set = defaultdict(set)
        for entity in data['vertexSet']:
            for mention in entity:
                if 'Q' in mention:
                    ent_id = 'Q' + str(mention['Q'])
                    passage_id = mention['pos'][0]
                    ent_set[ent_id].add(passage_id)
        for key, value in ent_set.items():
            ent_set[key] = list(value)
        redisd.set(f'doc-entities-{doc_title}', json.dumps(ent_set))
        pbar.update(1)
        line = f.readline()

  0%|          | 0/258079 [00:00<?, ?it/s]

# Loading distantly supervised data

In [4]:
def process(line):
    article = json.loads(line)
    tokens = list()
    mapping = dict()
    doc_id = int(article['id'])
    for para_id, para in enumerate(article['tokens']):
        for sent_id, sentence in enumerate(para):
            for word_id, word in enumerate(sentence):
                subwords = tokenizer.tokenize(word)
                mapping[(para_id, sent_id, word_id)] = list(range(len(tokens), len(tokens) + len(subwords)))
                tokens.extend(subwords)
    qs = list()
    for entity in article['vertexSet']:
        spans = list()
        for mention in entity:
            if 'Q' in mention:
                subwords = list()
                for position in range(mention['pos'][2], mention['pos'][3]):
                    subwords.extend(mapping[(mention['pos'][0], mention['pos'][1], position)])
                span = [min(subwords), max(subwords) + 1]
                spans.append(span)
        if len(spans) == len(entity):
            qs.append({
                'Q': entity[0]['Q'],
                'spans': spans
            })
        else:
            qs.append(None)
    instances = list()
    kset = set()
    for edge in article['edgeSet']:
        h = edge['h']
        t = edge['t']
        kset.add((h, t))
        if qs[h] is None or qs[t] is None:
            continue
        for r in edge['rs']:
            if 'P' + str(r) in relations:
                span_h = qs[h]['spans'][0]
                span_t = qs[t]['spans'][0]
                instances.append([doc_id, span_h[0], span_h[1], span_t[0], span_t[1], 'P' + str(r)])
    no_relations = list()
    for i in range(len(qs)):
        if qs[i] is None:
            continue
        for j in range(len(qs)):
            if qs[j] is None:
                continue
            if i != j and (i, j) not in kset:
                no_relations.append((i, j))
    if len(no_relations) > len(instances):
        no_relations = random.choices(no_relations, k=len(instances))
    for i, j in no_relations:
        instances.append([doc_id, qs[i]['spans'][0][0], qs[i]['spans'][0][1], qs[j]['spans'][0][0], qs[j]['spans'][0][1], 'n/a'])
    redisd.set(f'dsre-doc-{doc_id}', json.dumps(tokens))
    return instances, article['title'] in dev_docs

In [5]:
def initializer(base_model, _relations, t_docs):
    global redisd
    global tokenizer
    global relations
    global dev_docs
    redisd = redis.Redis(host='localhost', port=6379, decode_responses=True)
    tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
    relations = set(_relations)
    dev_docs = t_docs

In [6]:
dev_dataset = json.load(open('../data/rawdata/dev_dataset.json'))
dev_docs = set(map(lambda x: x[1], dev_dataset)) | set(map(lambda x: x[2], dev_dataset))

relations = json.load(open('../data/rawdata/relations.json'))
lines = list()
print("Loading distantly supervised documents...")
with open('../data/rawdata/distant_documents.jsonl') as f:
    for line in tqdm(f):
        lines.append(line.strip())
train_examples = list()
dev_examples = list()
print("Processing and caching to redis...")
with Pool(48, initializer=initializer, initargs=('bert-base-cased', relations, dev_docs)) as p:
    for instances, is_dev in tqdm(p.imap(process, lines)):
        if is_dev:
            dev_examples.extend(instances)
        else:
            train_examples.extend(instances)
json.dump(train_examples, open('../data/dsre_train_examples.json', 'w'))
json.dump(dev_examples, open('../data/dsre_dev_examples.json', 'w'))

Loading distantly supervised documents...


0it [00:00, ?it/s]

Processing and caching to redis...


0it [00:00, ?it/s]

# Caching documents to redis

In [7]:
def process(line):
    line = line.strip()
    if len(line) == 0:
        return None
    article = json.loads(line)
    tokens = list()
    mapping = dict()
    doc_id = int(article['id'])
    passage_mapping = []
    sentence_mapping = []
    for para_id, para in enumerate(article['tokens']):
        sentence_mapping.append([])
        for sent_id, sentence in enumerate(para):
            for word_id, word in enumerate(sentence):
                subwords = tokenizer.tokenize(word)
                mapping[(para_id, sent_id, word_id)] = list(range(len(tokens), len(tokens) + len(subwords)))
                tokens.extend(subwords)
            sentence_mapping[-1].append(len(tokens))
        passage_mapping.append(len(tokens))
    
    qs = list()
    for entity in article['vertexSet']:
        assert len(entity) > 0
        spans = list()
        for mention in entity:
            subwords = list()
            for position in range(mention['pos'][2], mention['pos'][3]):
                k = (mention['pos'][0], mention['pos'][1], position)
                if k in mapping:
                    subwords.extend(mapping[k])
            if len(subwords) > 0:
                span = [min(subwords), max(subwords) + 1, mention['pos'][0], mention['pos'][1]]
                spans.append(span)
        if len(spans) > 0:
            k = dict()
            for key in entity[0]:
                if key != 'pos':
                    k[key] = entity[0][key]
                    k['spans'] = spans
            qs.append(k)
    obj = dict()
    obj['tokens'] = tokens
    obj['entities'] = qs
    obj['id'] = article['id']
    obj['title'] = article['title']
    obj['passage_mapping'] = passage_mapping
    obj['sentence_mapping'] = sentence_mapping
    redisd.set(f'codred-doc-open-{obj["title"]}', json.dumps(obj))
    return doc_id, article['title']

In [8]:
def initializer(base_model):
    global redisd
    global tokenizer
    redisd = redis.Redis(host='localhost', port=6379, decode_responses=True)
    tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)

In [9]:
redisd = redis.Redis(host='localhost', port=6379, decode_responses=True)

popular_ids = list()
with open('../data/rawdata/popular_page_ent_link.jsonl') as f:
    with Pool(48, initializer=initializer, initargs=('bert-base-cased',)) as p:
        for doc_id, title in tqdm(p.imap_unordered(process, f)):
            popular_ids.append([doc_id, title])
json.dump(popular_ids, open('popular_docs.json', 'w'))

0it [00:00, ?it/s]

# Build entity doc mapping for open setting

In [9]:
doc_keys = [key for key in redisd.keys() if key.startswith("codred-doc-")]

In [14]:
raw_data_path = "../data/rawdata/"
with open(os.path.join(raw_data_path, "train_dataset.json")) as f:
    train_data = json.load(f)
with open(os.path.join(raw_data_path, "dev_dataset.json")) as f:
    dev_data = json.load(f)
with open(os.path.join(raw_data_path, "CodRED_test_dataset_in_closed_setting.json")) as f:
    test_data = json.load(f)
with open(os.path.join(raw_data_path, "CodRED_test_dataset_in_open_setting.json")) as f:
    test_open_data = json.load(f)
with open(os.path.join(raw_data_path, "train_evi.json")) as f:
    train_evi = json.load(f)
with open(os.path.join(raw_data_path, "dev_evi.json")) as f:
    dev_evi = json.load(f)

In [30]:
# process test data to inference formation
test_data_processed = []
for each in test_data:
    test_data_processed.append(["#".join([each['h_id'], each['t_id']]), each['doc'][0], each['doc'][1], 'n/a'])
with open(os.path.join(raw_data_path, "test_dataset_closed.json"), "w") as f:
    json.dump(test_data_processed, f)

In [31]:
# load processed test data
with open(os.path.join(raw_data_path, "test_dataset_closed.json")) as f:
    test_data = json.load(f)

In [36]:
# collect all entities
all_entities = set()
for each in (train_data + dev_data + test_data):
    head, tail = each[0].split("#")
    all_entities.add(head)
    all_entities.add(tail)
for each in (train_evi + dev_evi):
    head, tail = each['key'].split("#")
    all_entities.add(head)
    all_entities.add(tail)
all_entities = list(all_entities)

In [37]:
with open("../data/all_entities.json", "w") as f:
    json.dump(all_entities, f)

In [39]:
# add entities set in redis: {ent_id: list of documents containing this entity}
all_entities = set(all_entities)
for doc_key in tqdm(doc_keys):
    doc = json.loads(redisd.get(doc_key))
    entities = doc['entities']
    for ent in entities:
        if 'Q' in ent:
            ent_id = 'Q' + str(ent['Q'])
            if ent_id in all_entities:
                redisd.sadd(ent_id, doc_key)


  3%|████▉                                                                                                                                                           | 7961/258079 [00:08<04:49, 863.08it/s]


  6%|█████████▍                                                                                                                                                     | 15391/258079 [00:17<08:33, 472.32it/s]


  9%|██████████████▍                                                                                                                                               | 23588/258079 [00:26<03:44, 1045.08it/s]


 12%|██████████████████▉                                                                                                                                            | 30792/258079 [00:34<05:12, 727.08it/s]


 15%|███████████████████████▋                                                                                                                                       | 38540/258079 [00:43<03:50, 952.51it/s]


...

 89%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                 | 230371/258079 [04:12<00:29, 936.77it/s]


 92%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉            | 238309/258079 [04:21<00:21, 910.81it/s]


 95%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎       | 245584/258079 [04:30<00:15, 811.62it/s]


 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████   | 253371/258079 [04:39<00:05, 826.84it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 258079/258079 [04:44<00:00, 907.01it/s]


# open setting extraction

## Open setting initial filtering experiments

In [2]:
with open("../data/rawdata/dev_evi.json") as f:
    dev_evi = json.load(f)

In [41]:
dev_evi_open = defaultdict(list)
for each in dev_evi:
    dev_evi_open[tuple(each['key'].split("#"))].append(each)

In [42]:
dev_evi_open_true = defaultdict(lambda: [set(), set()])
for key in dev_evi_open:
    for record in dev_evi_open[key]:
        dev_evi_open_true[key][0].add(record['doc_h'])
        dev_evi_open_true[key][1].add(record['doc_t'])

In [43]:
def get_cand_doc(redisd, ent):
    return [each.replace("codred-doc-open-", "") for each in redisd.smembers(ent)]

In [44]:
def get_support_passages(redisd, h, t, title_h, title_t):
    ent_h = json.loads(redisd.get(f"doc-entities-{title_h}"))
    ent_t = json.loads(redisd.get(f"doc-entities-{title_t}"))
    shared_ent = set(ent_h.keys()).intersection(set(ent_t.keys()))
    if len(shared_ent) == 0:
        return None
    
    h_passages = [title_h + "_" + str(each) for each in ent_h[h]]
    t_passages = [title_t + "_" + str(each) for each in ent_t[t]]
    shared_passages = []
    for ent in shared_ent:
        shared_passages += [title_h + "_" + str(each) for each in ent_h[ent]]
        shared_passages += [title_t + "_" + str(each) for each in ent_t[ent]]
    return h_passages, t_passages, shared_passages

In [45]:
def rank_doc(redisd, h, t, cand_doc_h, cand_doc_t, topk=16):
    cand_doc_pairs = []
    for title_h in cand_doc_h:
        for title_t in cand_doc_t:
            support_passages = get_support_passages(redisd, h, t, title_h, title_t)
            if support_passages:
                heappush(cand_doc_pairs, (-len(support_passages[1]), (title_h, title_t)))
    if len(cand_doc_pairs) > topk:
        cand_doc_pairs_topk = []
        for i in range(topk):
            cand_doc_pairs_topk.append(heappop(cand_doc_pairs)[1])
        return cand_doc_pairs_topk
    return [each[1] for each in cand_doc_pairs]

In [46]:
def retrieve_doc_pairs(entities):
    h, t = entities
    cand_doc_h, cand_doc_t = get_cand_doc(redisd, h), get_cand_doc(redisd, t)
    cand_doc_pairs_topk = rank_doc(redisd, h, t, cand_doc_h, cand_doc_t)
    return (h, t), cand_doc_pairs_topk

def retrieve_doc_pairs_infer(sample):
    (h, t), r = sample
    cand_doc_h, cand_doc_t = get_cand_doc(redisd, h), get_cand_doc(redisd, t)
    cand_doc_pairs_topk = rank_doc(redisd, h, t, cand_doc_h, cand_doc_t)
    return (h, t, r), cand_doc_pairs_topk

In [49]:
dev_evi_open_input = list(dev_evi_open.keys())

def initializer():
    global redisd
    redisd = redis.Redis(host='localhost', port=6379, decode_responses=True)

doc_pairs_ids = list()
with Pool(48, initializer=initializer) as p:
    for record in tqdm(p.imap_unordered(retrieve_doc_pairs, dev_evi_open_input)):
        doc_pairs_ids.append(record)


0it [00:00, ?it/s]
1it [00:00,  4.14it/s]
4it [00:00, 12.75it/s]
8it [00:00, 21.69it/s]
12it [00:00, 26.64it/s]
16it [00:00, 17.05it/s]
20it [00:01, 21.39it/s]
23it [00:01, 14.98it/s]
26it [00:01, 13.84it/s]
29it [00:01, 14.71it/s]
32it [00:01, 17.17it/s]
35it [00:02, 10.69it/s]
37it [00:02, 10.17it/s]
40it [00:02, 12.81it/s]
44it [00:03, 14.27it/s]
46it [00:03, 14.97it/s]
49it [00:03, 17.18it/s]
52it [00:04,  8.44it/s]
54it [00:04,  7.94it/s]
57it [00:04,  9.93it/s]
59it [00:04,  9.82it/s]
62it [00:04, 11.62it/s]
64it [00:05, 10.42it/s]
67it [00:05,  8.61it/s]
70it [00:05, 10.77it/s]
72it [00:06,  4.54it/s]
74it [00:07,  5.24it/s]
76it [00:07,  4.00it/s]
77it [00:08,  4.40it/s]
78it [00:08,  4.68it/s]
79it [00:08,  4.15it/s]
80it [00:08,  3.78it/s]
83it [00:09,  4.68it/s]
84it [00:10,  2.83it/s]
85it [00:11,  2.03it/s]
87it [00:11,  3.07it/s]
88it [00:11,  3.34it/s]
89it [00:12,  1.91it/s]
90it [00:13,  2.16it/s]
91it [00:13,  2.32it/s]
92it [00:14,  2.24it/s]
93it [00:14,  2.45it/s]

In [52]:
cnt = 0
for ents, doc_pairs in doc_pairs_ids:
    trues = dev_evi_open_true[ents]
    doc_h, doc_t = set(), set()
    for dh, dt in doc_pairs:
        doc_h.add(dh)
        doc_t.add(dt)
    if len(trues[0].intersection(doc_h)) > 0 and len(trues[1].intersection(doc_t)) > 0:
        cnt += 1
cnt/len(doc_pairs_ids)

0.5546218487394958

## Open setting initial filtering inference

In [59]:
def extract_open_data(dataset):
    dataset_open = defaultdict(set)
    for each in dataset:
        dataset_open[tuple(each[0].split("#"))].add(each[3])
    for key in dataset_open:
        relations = list(dataset_open[key])
        if len(relations) == 1 and relations[0] == 'n/a':
            dataset_open[key] = 'n/a'
        else:
            for relation in relations:
                if relation != 'n/a':
                    dataset_open[key] = relation
                    break
    dataset_open = dataset_open.items()
    return dataset_open

In [62]:
dev_open = extract_open_data(dev_data)
test_open = extract_open_data(test_data)

In [65]:
def initializer():
    global redisd
    redisd = redis.Redis(host='localhost', port=6379, decode_responses=True)

doc_pairs_ids = list()
with Pool(48, initializer=initializer) as p:
    for record in tqdm(p.imap_unordered(retrieve_doc_pairs_infer, dev_open)):
        doc_pairs_ids.append(record)


0it [00:00, ?it/s]
1it [00:00,  3.51it/s]
3it [00:00,  8.30it/s]
8it [00:00, 19.71it/s]
12it [00:00, 24.18it/s]
15it [00:00, 16.16it/s]
20it [00:01, 20.51it/s]
23it [00:01, 14.52it/s]
27it [00:01, 16.21it/s]
30it [00:01, 17.86it/s]
33it [00:02, 12.65it/s]
35it [00:02, 12.24it/s]
39it [00:02, 15.43it/s]
41it [00:02, 14.84it/s]
45it [00:02, 17.51it/s]
49it [00:02, 20.82it/s]
52it [00:03,  8.35it/s]
54it [00:04,  8.03it/s]
56it [00:04,  6.04it/s]
58it [00:05,  6.80it/s]
61it [00:05,  7.10it/s]
62it [00:05,  7.27it/s]
63it [00:06,  4.64it/s]
64it [00:06,  5.15it/s]
65it [00:06,  3.55it/s]
66it [00:07,  3.32it/s]
67it [00:07,  3.43it/s]
71it [00:07,  6.98it/s]
73it [00:08,  4.86it/s]
75it [00:08,  4.88it/s]
76it [00:08,  5.12it/s]
77it [00:09,  5.18it/s]
80it [00:09,  5.32it/s]
81it [00:10,  3.18it/s]
82it [00:10,  2.99it/s]
83it [00:11,  2.32it/s]
84it [00:11,  2.85it/s]
85it [00:11,  3.27it/s]
86it [00:12,  3.95it/s]
87it [00:13,  1.95it/s]
88it [00:13,  2.17it/s]
89it [00:14,  1.73it/s]

1247it [03:40,  1.78it/s]
1248it [03:40,  2.04it/s]
1249it [03:40,  2.40it/s]
1250it [03:41,  2.39it/s]
1251it [03:41,  2.78it/s]
1254it [03:42,  3.19it/s]
1255it [03:42,  3.49it/s]
1257it [03:43,  3.04it/s]
1258it [03:43,  3.29it/s]
1259it [03:43,  2.94it/s]
1261it [03:45,  1.98it/s]
1262it [03:45,  2.17it/s]
1263it [03:46,  1.95it/s]
1264it [03:47,  1.80it/s]
1265it [03:47,  2.01it/s]
1266it [03:47,  2.45it/s]
1267it [03:48,  1.60it/s]
1270it [03:49,  2.73it/s]
1271it [03:49,  3.14it/s]
1274it [03:49,  5.20it/s]
1277it [03:49,  7.42it/s]
1280it [03:49, 10.20it/s]
1282it [03:49, 11.36it/s]
1284it [03:50,  9.49it/s]
1286it [03:50,  7.50it/s]
1288it [03:50,  7.62it/s]
1290it [03:51,  4.24it/s]
1294it [03:52,  6.34it/s]
1296it [03:52,  6.05it/s]
1298it [03:52,  5.42it/s]
1300it [03:53,  5.01it/s]
1301it [03:53,  4.82it/s]
1302it [03:53,  5.08it/s]
1304it [03:54,  4.97it/s]
1305it [03:54,  5.50it/s]
1307it [03:54,  6.88it/s]
1308it [03:54,  5.28it/s]
1310it [03:55,  6.15it/s]
1312it [03:5

2923it [06:08, 29.42it/s]
2927it [06:09, 29.24it/s]
2931it [06:09, 31.31it/s]
2936it [06:09, 32.52it/s]
2940it [06:09, 30.48it/s]
2944it [06:09, 30.64it/s]
2948it [06:09, 19.80it/s]
2951it [06:10, 19.54it/s]
2954it [06:10, 19.56it/s]
2957it [06:10, 18.83it/s]
2960it [06:10, 20.53it/s]
2966it [06:10, 26.25it/s]
2969it [06:10, 25.57it/s]
2972it [06:11, 22.77it/s]
2976it [06:11, 22.80it/s]
2983it [06:11, 32.19it/s]
2987it [06:11, 25.09it/s]
2990it [06:11, 24.55it/s]
2993it [06:11, 25.67it/s]
2996it [06:11, 23.97it/s]
3000it [06:12, 25.57it/s]
3006it [06:12, 32.45it/s]
3010it [06:12, 33.62it/s]
3014it [06:12, 34.99it/s]
3018it [06:12, 30.86it/s]
3022it [06:12, 32.23it/s]
3029it [06:12, 41.66it/s]
3034it [06:12, 33.05it/s]
3043it [06:13, 44.10it/s]
3048it [06:13, 44.94it/s]
3053it [06:13, 29.67it/s]
3057it [06:13, 29.87it/s]
3062it [06:14, 19.02it/s]
3065it [06:14, 12.50it/s]
3068it [06:14, 12.79it/s]
3070it [06:15, 13.31it/s]
3072it [06:15, 11.36it/s]
3074it [06:15, 12.26it/s]
3076it [06:1

4962it [08:12, 25.48it/s]
4965it [08:12, 21.82it/s]
4969it [08:12, 21.62it/s]
4972it [08:12, 22.69it/s]
4975it [08:12, 24.18it/s]
4978it [08:12, 19.97it/s]
4981it [08:13, 18.76it/s]
4984it [08:13, 16.29it/s]
4988it [08:13, 16.73it/s]
4990it [08:13, 14.66it/s]
4992it [08:13, 13.95it/s]
4999it [08:14, 22.73it/s]
5009it [08:14, 32.56it/s]
5013it [08:14, 30.36it/s]
5017it [08:14, 18.05it/s]
5021it [08:15, 19.90it/s]
5024it [08:15, 14.62it/s]
5028it [08:15, 13.35it/s]
5032it [08:15, 16.15it/s]
5035it [08:16, 15.91it/s]
5046it [08:16, 28.50it/s]
5050it [08:16, 29.33it/s]
5054it [08:16, 25.53it/s]
5059it [08:16, 27.64it/s]
5063it [08:17, 21.12it/s]
5068it [08:17, 25.04it/s]
5072it [08:17, 26.40it/s]
5076it [08:17, 15.38it/s]
5079it [08:17, 17.14it/s]
5082it [08:18, 14.99it/s]
5091it [08:18, 25.78it/s]
5095it [08:18, 27.65it/s]
5099it [08:18, 29.11it/s]
5103it [08:18, 30.26it/s]
5107it [08:18, 29.31it/s]
5111it [08:19, 17.61it/s]
5114it [08:19, 17.53it/s]
5117it [08:19, 12.40it/s]
5119it [08:2

In [66]:
output = []
for (h, t, r), doc_pairs in doc_pairs_ids:
    for doc_h, doc_t in doc_pairs:
        output.append(["#".join([h,t]), doc_h, doc_t, r])
with open("../data/open_setting_data/dev_data_shared_entities_ranked.json", "w") as f:
    json.dump(output, f)

In [67]:
def initializer():
    global redisd
    redisd = redis.Redis(host='localhost', port=6379, decode_responses=True)

doc_pairs_ids = list()
with Pool(48, initializer=initializer) as p:
    for record in tqdm(p.imap_unordered(retrieve_doc_pairs_infer, test_open)):
        doc_pairs_ids.append(record)


0it [00:00, ?it/s]
1it [00:00,  4.09it/s]
4it [00:00, 12.31it/s]
10it [00:00, 26.83it/s]
14it [00:00, 29.73it/s]
20it [00:00, 35.93it/s]
24it [00:00, 34.85it/s]
28it [00:01, 26.38it/s]
32it [00:01, 13.37it/s]
35it [00:02, 10.32it/s]
37it [00:02,  9.59it/s]
40it [00:02, 10.67it/s]
42it [00:02, 11.22it/s]
44it [00:02, 11.72it/s]
46it [00:03, 12.08it/s]
48it [00:04,  5.20it/s]
50it [00:04,  6.01it/s]
52it [00:04,  6.18it/s]
53it [00:04,  5.51it/s]
55it [00:05,  6.58it/s]
56it [00:05,  6.85it/s]
57it [00:05,  4.63it/s]
58it [00:05,  4.73it/s]
59it [00:06,  2.42it/s]
61it [00:07,  3.16it/s]
62it [00:07,  3.45it/s]
63it [00:07,  3.97it/s]
65it [00:07,  4.58it/s]
66it [00:08,  4.04it/s]
68it [00:08,  5.46it/s]
69it [00:08,  5.77it/s]
71it [00:08,  5.84it/s]
72it [00:09,  5.95it/s]
73it [00:09,  5.52it/s]
76it [00:10,  3.56it/s]
77it [00:10,  3.59it/s]
78it [00:12,  1.80it/s]
79it [00:12,  2.04it/s]
80it [00:13,  1.59it/s]
81it [00:13,  1.93it/s]
82it [00:15,  1.38it/s]
83it [00:15,  1.42it/s

1099it [04:27,  5.37it/s]
1101it [04:27,  4.87it/s]
1102it [04:27,  4.76it/s]
1103it [04:28,  3.35it/s]
1105it [04:28,  4.18it/s]
1107it [04:28,  5.66it/s]
1108it [04:29,  4.71it/s]
1111it [04:29,  7.17it/s]
1113it [04:29,  6.62it/s]
1114it [04:29,  6.83it/s]
1115it [04:30,  6.96it/s]
1116it [04:30,  5.81it/s]
1117it [04:32,  1.69it/s]
1118it [04:32,  2.02it/s]
1119it [04:33,  1.71it/s]
1120it [04:33,  2.07it/s]
1122it [04:33,  2.65it/s]
1124it [04:34,  3.62it/s]
1125it [04:35,  2.33it/s]
1126it [04:36,  1.35it/s]
1128it [04:38,  1.22it/s]
1129it [04:38,  1.49it/s]
1130it [04:39,  1.69it/s]
1131it [04:39,  1.90it/s]
1132it [04:40,  1.44it/s]
1133it [04:41,  1.31it/s]
1134it [04:42,  1.24it/s]
1135it [04:43,  1.42it/s]
1137it [04:43,  2.05it/s]
1138it [04:44,  2.11it/s]
1140it [04:44,  2.64it/s]
1142it [04:44,  3.86it/s]
1143it [04:45,  3.00it/s]
1144it [04:45,  3.08it/s]
1145it [04:46,  1.72it/s]
1148it [04:47,  2.88it/s]
1150it [04:47,  2.87it/s]
1151it [04:48,  3.20it/s]
1152it [04:4

2536it [07:14, 23.04it/s]
2539it [07:14, 21.50it/s]
2542it [07:14, 16.62it/s]
2544it [07:14, 16.70it/s]
2546it [07:14, 16.02it/s]
2548it [07:14, 15.58it/s]
2550it [07:15, 12.85it/s]
2552it [07:15, 13.89it/s]
2554it [07:15, 11.34it/s]
2559it [07:15, 16.75it/s]
2563it [07:15, 19.12it/s]
2566it [07:16, 17.42it/s]
2568it [07:16, 12.55it/s]
2571it [07:16, 14.50it/s]
2574it [07:16, 13.73it/s]
2576it [07:16, 12.66it/s]
2580it [07:17, 16.48it/s]
2584it [07:17, 17.52it/s]
2591it [07:17, 26.78it/s]
2595it [07:17, 21.27it/s]
2598it [07:17, 20.48it/s]
2601it [07:17, 19.45it/s]
2604it [07:18, 19.15it/s]
2612it [07:18, 30.50it/s]
2616it [07:18, 29.47it/s]
2620it [07:18, 20.08it/s]
2624it [07:18, 21.02it/s]
2629it [07:19, 24.84it/s]
2633it [07:19, 25.32it/s]
2636it [07:19, 22.39it/s]
2643it [07:19, 28.11it/s]
2647it [07:19, 20.04it/s]
2650it [07:20, 20.27it/s]
2654it [07:20, 21.97it/s]
2662it [07:20, 32.18it/s]
2666it [07:20, 27.44it/s]
2670it [07:20, 20.48it/s]
2673it [07:21, 14.99it/s]
2677it [07:2

4449it [09:14, 22.72it/s]
4458it [09:14, 36.87it/s]
4463it [09:14, 39.18it/s]
4471it [09:14, 48.95it/s]
4477it [09:14, 35.44it/s]
4482it [09:15, 30.91it/s]
4493it [09:15, 44.45it/s]
4499it [09:15, 40.38it/s]
4504it [09:15, 37.26it/s]
4509it [09:15, 35.49it/s]
4516it [09:15, 39.37it/s]
4521it [09:16, 39.61it/s]
4526it [09:16, 40.53it/s]
4531it [09:16, 29.98it/s]
4535it [09:16, 24.10it/s]
4539it [09:16, 24.27it/s]
4544it [09:17, 28.83it/s]
4548it [09:17, 29.50it/s]
4552it [09:17, 25.63it/s]
4557it [09:17, 29.67it/s]
4561it [09:17, 29.93it/s]
4572it [09:17, 39.52it/s]
4581it [09:17, 49.21it/s]
4587it [09:18, 42.85it/s]
4594it [09:18, 46.46it/s]
4600it [09:18, 49.40it/s]
4606it [09:18, 44.50it/s]
4611it [09:18, 41.32it/s]
4616it [09:18, 40.57it/s]
4621it [09:19, 30.66it/s]
4626it [09:19, 32.66it/s]
4633it [09:19, 39.19it/s]
4641it [09:19, 48.09it/s]
4650it [09:19, 57.48it/s]
4657it [09:19, 52.02it/s]
4663it [09:19, 48.32it/s]
4669it [09:19, 40.95it/s]
4674it [09:20, 34.16it/s]
4679it [09:2

In [68]:
output = []
for (h, t, r), doc_pairs in doc_pairs_ids:
    for doc_h, doc_t in doc_pairs:
        output.append(["#".join([h,t]), doc_h, doc_t, r])
with open("../data/open_setting_data/test_data_shared_entities_ranked.json", "w") as f:
    json.dump(output, f)

# close setting extraction

In [4]:
raw_data_path = "../data/rawdata/"
with open(os.path.join(raw_data_path, "train_dataset.json")) as f:
    train_data = json.load(f)
with open(os.path.join(raw_data_path, "dev_dataset.json")) as f:
    dev_data = json.load(f)
with open(os.path.join(raw_data_path, "test_dataset_closed.json")) as f:
    test_data = json.load(f)
with open("../data/open_setting_data/dev_data_shared_entities_ranked.json") as f:
    dev_open_data = json.load(f)
with open("../data/open_setting_data/test_data_shared_entities_ranked.json") as f:
    test_open_data = json.load(f)

In [74]:
with open(os.path.join(raw_data_path, "train_evi.json")) as f:
    train_evi = json.load(f)
with open(os.path.join(raw_data_path, "dev_evi.json")) as f:
    dev_evi = json.load(f)

In [5]:
with open("../data/q2name.json") as f:
    q2name = json.load(f)

In [6]:
def load_sample(sample):
    if isinstance(sample, list):
        ht, doch_title, doct_title, r = sample
        h,t = ht.split("#")
        return h, t, doch_title, doct_title, r, None, None
    else:
        h,t = sample['key'].split("#")
        r = sample['r']
        doch_title = sample['doc_h']
        doct_title = sample['doc_t']
        evis_h = sample['evis_h']
        evis_t = sample['evis_t']
        return h, t, doch_title, doct_title, r, evis_h, evis_t

In [7]:
def build_reverse_idx(doch, doct):
    doch_title = doch['title']
    doct_title = doct['title']
    
    entity_reverse_idx = {}
    doc_reverse_idx = {}
    for entity in doch['entities']:
        if 'Q' in entity:
            entity_idx = 'Q' + str(entity['Q'])
            for span in entity['spans']:
                passage_idx = doch_title + "_" + str(span[2])
                if entity_idx not in entity_reverse_idx:
                    entity_reverse_idx[entity_idx] = set()
                entity_reverse_idx[entity_idx].add(passage_idx)
                if passage_idx not in doc_reverse_idx:
                    doc_reverse_idx[passage_idx] = set()
                doc_reverse_idx[passage_idx].add(entity_idx)
            
    for entity in doct['entities']:
        if 'Q' in entity:
            entity_idx = 'Q' + str(entity['Q'])
            for span in entity['spans']:
                passage_idx = doct_title + "_" + str(span[2])
                if entity_idx not in entity_reverse_idx:
                    entity_reverse_idx[entity_idx] = set()
                entity_reverse_idx[entity_idx].add(passage_idx)
                if passage_idx not in doc_reverse_idx:
                    doc_reverse_idx[passage_idx] = set()
                doc_reverse_idx[passage_idx].add(entity_idx)
    return entity_reverse_idx, doc_reverse_idx

In [8]:
def get_neighbor(passage_id, path_entities, entity_reverse_idx, doc_reverse_idx):
    if "_" not in passage_id:
        next_passages = entity_reverse_idx[passage_id]
        return set(zip(next_passages, [passage_id] * len(next_passages)))
    else:
        shared_entities = doc_reverse_idx[passage_id].difference(path_entities)
        output = []
        for entity in shared_entities:
            for next_passage in entity_reverse_idx[entity]:
                if next_passage != passage_id:
                    output.append((next_passage, entity))
        return set(output)

In [9]:
def findAllPath(
        h, t,
        doch, doct,
        max_step=3,
        max_iteration=1e5
    ):
    
    doch_title = doch['title']
    doct_title = doct['title']
    
    entity_reverse_idx, doc_reverse_idx = build_reverse_idx(doch, doct)
    
    end_set = entity_reverse_idx[t]
    path = []
    stack = []
    visited = set()
    seen_path = {}
    stack.append(h)
    visited.add(h)
    paths_entities = [None]
    seen_path[h] = []
    
    iter_num = 0
    while len(stack) > 0 and iter_num < max_iteration:
        iter_num += 1
        
        start = stack[-1]
        nodes = get_neighbor(start, set(paths_entities), entity_reverse_idx, doc_reverse_idx)
        if start.startswith(doct_title):
            nodes = [each for each in nodes if each[0].startswith(doct_title)]
        if start not in seen_path.keys():
            seen_path[start] = []
        g = 0
        for w, ent in nodes:
            if w not in visited and w not in seen_path[start]:
                g = g+1
                stack.append(w)
                paths_entities.append(ent)
                visited.add(w)
                seen_path[start].append(w)
                if w in end_set:
                    path.append(list(zip(list(stack), list(paths_entities))))
                    old_pop = stack.pop()
                    paths_entities.pop()
                    visited.remove(old_pop)
                break
        if g == 0 or len(stack) > max_step:
            old_pop = stack.pop()
            paths_entities.pop()
            if old_pop in seen_path:
                del seen_path[old_pop]
            visited.remove(old_pop)
            
    success = len(path) > 0
    if not success:
        path = [(h, None)]
        start_list = list(entity_reverse_idx[h])
        end_list = list(end_set)
        if len(start_list) + len(end_list) <= max_step:
            path += [(each, h) for each in start_list] + [(each, t) for each in end_list]
        else:
            start_length = max_step // 2
            end_length = max_step - start_length
            path += [(each, h) for each in start_list[:start_length]] + [(each, t) for each in end_list[:end_length]]
        path = [path]
    return path, success

In [144]:
h, t, doch_title, doct_title, r, evis_h, evis_t = load_sample(dev_open_data[40700])
print(h, t, doch_title, doct_title, evis_h, evis_t)
doch = json.loads(redisd.get('codred-doc-open-'+doch_title))
doct = json.loads(redisd.get('codred-doc-open-'+doct_title))
findAllPath(h, t, doch, doct)[:10]

Q4466 Q2335128 RMS Titanic Colonel Tye None None


([[('Q4466', None),
   ('RMS Titanic_129', 'Q4466'),
   ('RMS Titanic_85', 'Q44578'),
   ('Colonel Tye_1', 'Q1164740')],
  [('Q4466', None),
   ('RMS Titanic_129', 'Q4466'),
   ('RMS Titanic_85', 'Q44578'),
   ('Colonel Tye_0', 'Q1164740')],
  [('Q4466', None),
   ('RMS Titanic_129', 'Q4466'),
   ('RMS Titanic_76', 'Q44578'),
   ('Colonel Tye_1', 'Q1164740')],
  [('Q4466', None),
   ('RMS Titanic_129', 'Q4466'),
   ('RMS Titanic_76', 'Q44578'),
   ('Colonel Tye_0', 'Q1164740')],
  [('Q4466', None),
   ('RMS Titanic_129', 'Q4466'),
   ('RMS Titanic_5', 'Q44578'),
   ('Colonel Tye_1', 'Q1164740')],
  [('Q4466', None),
   ('RMS Titanic_129', 'Q4466'),
   ('RMS Titanic_5', 'Q44578'),
   ('Colonel Tye_0', 'Q1164740')],
  [('Q4466', None),
   ('RMS Titanic_129', 'Q4466'),
   ('RMS Titanic_59', 'Q44578'),
   ('Colonel Tye_1', 'Q1164740')],
  [('Q4466', None),
   ('RMS Titanic_129', 'Q4466'),
   ('RMS Titanic_59', 'Q44578'),
   ('Colonel Tye_0', 'Q1164740')],
  [('Q4466', None),
   ('RMS Titan

In [27]:
def get_paths(sample):
    h, t, doch_title, doct_title, r, evis_h, evis_t = load_sample(sample)
    doch = json.loads(redisd.get('codred-doc-open-'+doch_title))
    doct = json.loads(redisd.get('codred-doc-open-'+doct_title))

    paths, success = findAllPath(h, t, doch, doct, max_step=4)
    return {'h':h, 't': t, 'r': r, 'h_name': q2name[h],'t_name': q2name[t],
            'paths': paths, 'doch_title': doch_title, 'doct_title': doct_title,
            'evis_h': evis_h, 'evis_t': evis_t, 'success': success}

In [13]:
test_all_paths = []
with Pool(48) as p:
    for paths in tqdm(p.imap_unordered(get_paths, test_data)):
        test_all_paths.append(paths)

40524it [02:02, 329.74it/s]


In [14]:
dev_all_paths = []
with Pool(48) as p:
    for paths in tqdm(p.imap_unordered(get_paths, dev_data)):
        dev_all_paths.append(paths)

40740it [02:04, 328.29it/s]


In [117]:
processed_set = set()
for each in dev_all_paths:
    processed_set.add((each['h'], each['t'], each['doch_title'], each['doct_title']))
for idx, each in enumerate(dev_data):
    if tuple(each[0].split("#")) + (each[1], each[2]) not in processed_set:
        print(idx, each)

29274 ['Q19520525#Q907357', 'List of Supernatural characters', 'Tron 2.0', 'n/a']


In [118]:
dev_data[29274]

['Q19520525#Q907357', 'List of Supernatural characters', 'Tron 2.0', 'n/a']

In [15]:
train_all_paths = []
with Pool(48) as p:
    for paths in tqdm(p.imap_unordered(get_paths, train_data)):
        train_all_paths.append(paths)

129548it [05:13, 412.83it/s]


In [18]:
dev_open_all_paths = []
with Pool(48) as p:
    for paths in tqdm(p.imap_unordered(get_paths, dev_open_data)):
        dev_open_all_paths.append(paths)

78023it [05:08, 253.22it/s] 


In [28]:
test_open_all_paths = []
with Pool(48) as p:
    for paths in tqdm(p.imap_unordered(get_paths, test_open_data)):
        test_open_all_paths.append(paths)

77940it [35:52, 36.21it/s] 


In [16]:
def eval_quality(all_paths):
    fail_cnt = 0
    avg_doc_path_num = 0
    avg_entity_path_num = 0
    for each in tqdm(all_paths):
        if not each['success']:
            fail_cnt += 1
        avg_doc_path_num += len(each['paths'])
        paths = [item[1:] for item in each['paths']]
        doc_paths = [[item[0] for item in path] for path in paths]
        ent_paths = set([tuple(item[1] for item in path) for path in paths])
        avg_entity_path_num += len(ent_paths)
    return {
        "fail rate": fail_cnt/len(all_paths),
        "avg_doc_path": avg_doc_path_num/len(all_paths),
        "avg_ent_path": avg_entity_path_num/len(all_paths)
    }

In [29]:
eval_quality(test_open_all_paths)

100%|██████████| 77940/77940 [02:30<00:00, 517.96it/s] 


{'fail rate': 0.16434436746215036,
 'avg_doc_path': 419.06244547087505,
 'avg_ent_path': 44.24531691044393}

In [20]:
with open("../data/doc_paths/test_data_3hop.json", "w") as f:
    for path in test_all_paths:
        f.write(json.dumps(path) + "\n")

In [22]:
with open("../data/doc_paths/dev_data_3hop.json", "w") as f:
    for path in dev_all_paths:
        f.write(json.dumps(path) + "\n")

In [23]:
with open("../data/doc_paths/train_data_3hop.json", "w") as f:
    for path in train_all_paths:
        f.write(json.dumps(path) + "\n")

In [24]:
with open("../data/doc_paths/dev_open_data_3hop.json", "w") as f:
    for path in dev_open_all_paths:
        f.write(json.dumps(path) + "\n")

In [30]:
with open("../data/doc_paths/test_open_data_4hop.json", "w") as f:
    for path in test_open_all_paths:
        f.write(json.dumps(path) + "\n")

In [26]:
!ls ../data/doc_paths

dev_data_3hop.json	 evi_data_3hop.json   test_data_4hop.json
dev_data_4hop.json	 evi_data_4hop.json   test_open_data_3hop.json
dev_open_data_3hop.json  evi_data_5hop.json   train_data_3hop.json
dev_open_data_4hop.json  evi_data.json	      train_data_4hop.json
evi_data_2hop.json	 test_data_3hop.json


## Evaluation in inference

In [46]:
recall_cnt = 0
fail_cnt = 0
avg_doc_path_num = 0
avg_entity_path_num = 0
for each in tqdm(all_paths):
    if not each['success']:
        fail_cnt += 1
    avg_doc_path_num += len(each['paths'])
    paths = [item[1:] for item in each['paths']]
    doc_paths = [[item[0] for item in path] for path in paths]
    ent_paths = set([tuple(item[1] for item in path) for path in paths])
    avg_entity_path_num += len(ent_paths)

    evis = []
    for evi in each['evis_h']:
        evis.append(each['doch_title'] + "_" + str(evi[0]))
    for evi in each['evis_t']:
        evis.append(each['doct_title'] + "_" + str(evi[0]))
    recall = 0
    for path in doc_paths:
        if set(evis).issubset(set(path)):
            recall = 1
    recall_cnt += recall

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3497/3497 [00:00<00:00, 12706.60it/s]


In [47]:
recall_cnt/len(all_paths), fail_cnt/len(all_paths), avg_doc_path_num/len(all_paths), avg_entity_path_num/len(all_paths)

(0.5842150414641121,
 0.057763797540749215,
 35.49699742636545,
 9.792965398913354)

In [56]:
recall_cnt/len(all_paths), fail_cnt/len(all_paths), avg_doc_path_num/len(all_paths), avg_entity_path_num/len(all_paths)

(0.7108950529024879, 0.04175007148984844, 411.0737775235917, 74.69831283957679)